In [12]:
import cv2
import numpy as np
import tensorflow as tf
from ultralyticsplus import YOLO
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import requests
from io import BytesIO
import os

# Force TensorFlow to use CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load YOLOv8 model and set to CPU
yolo_model = YOLO('foduucom/plant-leaf-detection-and-classification')
yolo_model.overrides['device'] = 'cpu'
yolo_model.overrides['conf'] = 0.25
yolo_model.overrides['iou'] = 0.45
yolo_model.overrides['agnostic_nms'] = False
yolo_model.overrides['max_det'] = 1000

# Load VGG16 model
vgg16_model = load_model("D:/Final_Year_project/Final_year_project_final/Vgg16/vgg16_All_final_model_1.h5")

# Define class labels for VGG16
vgg16_class_names = [
    'Groundnut_early_leaf_spot', 'Groundnut_early_rust', 'Groundnut_healthy_leaf',
    'Groundnut_late_leaf_spot', 'Groundnut_nutrition_deficiency', 'Groundnut_rust',
    'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight',
    'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites_Two-spotted_spider_mite', 'Tomato___Target_Spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy', 'potato_early', 'potato_healthy', 'potato_late'
]

# Preprocess image for VGG16
def preprocess_for_vgg16(cropped_img):
    resized_img = cv2.resize(cropped_img, (224, 224))
    array_img = img_to_array(resized_img)
    preprocessed_img = preprocess_input(np.expand_dims(array_img, axis=0))
    return preprocessed_img

# Function to load image from URL or local file
def load_image(image_path):
    if image_path.startswith("http"):
        response = requests.get(image_path)
        image = np.array(Image.open(BytesIO(response.content)))
        return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return cv2.imread(image_path)

# Detection and classification
def detect_and_classify(image_path):
    image = load_image(image_path)
    if image is None:
        print(f"Error: Could not load image at {image_path}")
        return

    # YOLO inference
    yolo_results = yolo_model.predict(source=image, device='cpu')

    # Extract bounding boxes
    detected_leaves = []
    for *box, conf, cls in yolo_results[0].boxes.data.tolist():
        x1, y1, x2, y2 = map(int, box)
        detected_leaves.append((x1, y1, x2, y2))

    if not detected_leaves:
        print("No leaves detected.")
        return

    print("Detected Diseases:")
    for i, (x1, y1, x2, y2) in enumerate(detected_leaves):
        cropped_leaf = image[y1:y2, x1:x2]
        vgg_input = preprocess_for_vgg16(cropped_leaf)
        predictions = vgg16_model.predict(vgg_input, verbose=0)
        predicted_class = vgg16_class_names[np.argmax(predictions)]
        confidence = np.max(predictions)

        # Print disease info in the terminal
        print(f"Leaf {i + 1}: {predicted_class} ({confidence:.2f})")

        # Draw bounding boxes and labels on the image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{predicted_class} ({confidence:.2f})", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Save the final image
    output_path = "output_image_full.jpg"
    cv2.imwrite(output_path, image)

    # Display the full image using matplotlib
    plt.figure(figsize=(12, 12))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Full Image with Detected Leaves and Diseases")
    plt.show()

    print(f"Processed image saved as {output_path}")

# Main function
def main():
    choice = input("Enter '1' for local image or '2' for an HTTPS image URL: ")
    if choice == '1':
        image_path = input("Enter the path to the local image: ").strip()
    elif choice == '2':
        image_path = input("Enter the HTTPS image URL: ").strip()
    else:
        print("Invalid choice.")
        return
    detect_and_classify(image_path)

if __name__ == "__main__":
    main()


12/30/2024 13:22:20 - WARNING - absl -   Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Ultralytics YOLOv8.0.43  Python-3.12.2 torch-2.5.1+cu118 CPU
Model summary (fused): 168 layers, 11143386 parameters, 0 gradients, 28.5 GFLOPs

0: 640x640 2 groundnuts, 166.9ms
Speed: 0.0ms preprocess, 166.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Detected Diseases:
Leaf 1: Groundnut_late_leaf_spot (1.00)
Leaf 2: Groundnut_nutrition_deficiency (0.98)
Processed image saved as output_image_full.jpg


C:\Users\Keert\AppData\Local\Temp\ipykernel_39780\2547154516.py:99: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
